In [2]:
using Oceananigans
using Oceananigans.Units

grid = RectilinearGrid(CPU(), size = (160, 32), extent = (10000meters, 500meters), topology = (Bounded, Flat, Bounded))

model = NonhydrostaticModel(; grid,
                              advection = WENO(; grid),
                              coriolis = FPlane(latitude = 0),
                              closure = AnisotropicMinimumDissipation(),
                              buoyancy = SeawaterBuoyancy(constant_salinity = true),
                              tracers = (:T, :S))

@inline Tᵢ(x, z) = 9 + 0.05 * tanh((x - 7000 + 4 * z) / 500)

set!(model, T = Tᵢ)

simulation = Simulation(model; Δt = 50, stop_time = 1days)

simulation.output_writers[:tracers] = NetCDFOutputWriter(model, model.tracers,
                                                       filename = "../data/buoyancy_front.nc",
                                                       schedule = TimeInterval(24minute),
                                                       overwrite_existing = true)

run!(simulation)

[ Info: Initializing simulation...
[ Info:     ... simulation initialization complete (13.866 ms)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (22.169 ms).
[ Info: Simulation is stopping after running for 21.024 seconds.
[ Info: Simulation time 1 day equals or exceeds stop time 1 day.
